# Debugging Model Training Performance using Profiler


## Learning objectives

1. Create a Cloud Storage bucket.
2. Create a TensorBoard instance.
3. Create and run a custom training job.
4. View the TensorBoard Profiler dashboard


## Overview

In this notebook, you will learn how to enable Vertex AI TensorBoard Profiler so you can debug model training performance for your custom training jobs.


### Dataset

The dataset used for this tutorial is the [mnist dataset](https://www.tensorflow.org/datasets/catalog/mnist) from [TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/overview).

Each learning objective will correspond to a __#TODO__ in the notebook, where you will complete the notebook cell's code before running the cell. Refer to the [solution notebook](../solutions/custom_training_tensorboard_profiler.ipynb) for reference.

### Install required packages

Install the following packages required to execute this notebook. 

- Ensure that you're using Tensorflow 2.4 or a later version.
- Install the Vertex AI SDK with the cloud-profiler plugin. From your local Docker container, run `pip install google-cloud-aiplatform[cloud_profiler]`


In [1]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

In [2]:
! pip3 install --upgrade tensorflow {USER_FLAG} -q
! pip3 install --upgrade google-cloud-aiplatform[cloud_profiler] {USER_FLAG} -q

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tfx-bsl 1.10.1 requires google-api-python-client<2,>=1.7.11, but you have google-api-python-client 2.63.0 which is incompatible.
tfx-bsl 1.10.1 requires pyarrow<7,>=6, but you have pyarrow 9.0.0 which is incompatible.
tensorflow-transform 1.10.1 requires pyarrow<7,>=6, but you have pyarrow 9.0.0 which is incompatible.
tensorflow-transform 1.10.1 requires tensorflow!=2.0.*,!=2.1.*,!=2.2.*,!=2.3.*,!=2.4.*,!=2.5.*,!=2.6.*,!=2.7.*,!=2.8.*,<2.10,>=1.15.5, but you have tensorflo

### Restart the kernel

After you install the required packages, you need to restart the notebook kernel so it can find the packages.

In [3]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### Set your project ID

**If you don't know your project ID**, you can get a list of projects available to you using `gcloud`.

In [1]:
import os

PROJECT_ID = "[your-project-id]" # Replace with your Project ID

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

Project ID:  qwiklabs-gcp-00-1fe7a1835a07


In [2]:
! gcloud config set project $PROJECT_ID

Updated property [core/project].


### Get your project number

Now that the project ID is set, you get your corresponding project number.

In [3]:
shell_output = ! gcloud projects list --filter="PROJECT_ID:'{PROJECT_ID}'" --format='value(PROJECT_NUMBER)'
PROJECT_NUMBER = shell_output[0]
print("Project number:", PROJECT_NUMBER)

Project number: 21417957250


### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. We recommend that you choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You may not use a multi-regional bucket for training with Vertex AI. Not all regions provide support for all Vertex AI services.

Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [4]:
REGION = "[your-region]"  # Replace with your region

if REGION == "[your-region]":
    REGION = "us-central1"

#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append it onto the name of resources you create in this tutorial.

In [5]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

#### Enable the Vertex AI API

In [6]:
! gcloud services enable aiplatform.googleapis.com

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

When you submit a training job using the Cloud SDK, you upload a Python package containing your training code to a Cloud Storage bucket. Vertex AI runs the code from this package. The Cloud Storage bucket is also used to store TensorBoard logs generated by your training jobs.

Set the name of your Cloud Storage bucket below. It must be unique across all Cloud Storage buckets.

In [7]:
BUCKET_NAME = "qwiklabs-gcp-00-1fe7a1835a07"  # Replace with your unique bucket name
BUCKET_URI = f"gs://{BUCKET_NAME}"

If you don't have a Cloud Storage bucket, create one by running the following cell:

In [8]:
# TODO 1
# Create a regional Cloud Storage bucket
# TODO 1: Your code goes here

Creating gs://qwiklabs-gcp-00-1fe7a1835a07/...


Finally, validate access to your Cloud Storage bucket by examining its contents:

In [9]:
! gsutil ls -al $BUCKET_URI

### Set up your service account

The Vertex AI TensorBoard integration with custom training requires attaching a service account with the Storage Admin role (roles/storage.admin) and Vertex AI User role (roles/aiplatform.user).

#### Set your service account

If you do not want to use your project's Compute Engine service account, set `SERVICE_ACCOUNT` to another service account ID.

In [27]:
SERVICE_ACCOUNT = "21417957250-compute@developer.gserviceaccount.com"  # Replace '[your-service-account]' with your project's Compute Engine service account

#### Set service account access for Vertex AI TensorBoard integration

Run the following commands to grant your service account access to the Storage Admin role (roles/storage.admin) and Vertex AI User role (roles/aiplatform.user)

In [10]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/aiplatform.user

! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.admin

### Import libraries

In [11]:
import google.cloud.aiplatform as aiplatform

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [12]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

#### Set hardware accelerators

You can set hardware accelerators for training.

Set the variables `TRAIN_GPU/TRAIN_NGPU` to use a container image supporting a GPU and the number of GPUs allocated to the virtual machine (VM) instance. For example, to use a GPU container image with 4 Nvidia Tesla K80 GPUs allocated to each VM, you would specify:

    (aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80, 4)

See the [locations where accelerators are available](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators).

Otherwise specify `(None, None)` to use a container image to run on a CPU.


In [13]:
if os.getenv("IS_TESTING_TRAIN_GPU"):
    TRAIN_GPU, TRAIN_NGPU = (
        aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_TRAIN_GPU")),
    )
else:
    TRAIN_GPU, TRAIN_NGPU = (None, None)

if os.getenv("IS_TESTING_DEPLOY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPLOY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

#### Set pre-built containers


Set the pre-built Docker container image for training and prediction.


For the latest list, see [Pre-built containers for training](https://cloud.google.com/ai-platform-unified/docs/training/pre-built-containers).

For the latest list, see [Pre-built containers for prediction](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers).

In [14]:
TF = "2-9"

if TRAIN_GPU:
    TRAIN_VERSION = "tf-gpu.{}".format(TF)
else:
    TRAIN_VERSION = "tf-cpu.{}".format(TF)

if DEPLOY_GPU:
    DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
else:
    DEPLOY_VERSION = "tf2-cpu.{}".format(TF)

TRAIN_IMAGE = "us-docker.pkg.dev/vertex-ai/training/{}:latest".format(TRAIN_VERSION)
DEPLOY_IMAGE = "us-docker.pkg.dev/vertex-ai/training/{}:latest".format(DEPLOY_VERSION)

print("Training image:", TRAIN_IMAGE, TRAIN_GPU, TRAIN_NGPU)
print("Deployment image:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

Training image: us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-9:latest None None
Deployment image: us-docker.pkg.dev/vertex-ai/training/tf2-cpu.2-9:latest None None


## Create a TensorBoard instance

A Vertex AI TensorBoard instance, which is a regionalized resource storing your Vertex AI TensorBoard experiments, must be created before the experiments can be visualized. You can create multiple instances in a project. You can use command  `gcloud ai tensorboards list` to get a list of your existing TensorBoard instances.

#### Set your TensorBoard instance display name


In [15]:
TENSORBOARD_NAME = "tensorboard-tb"  # Replace with any name of your choice

If you don't have a TensorBoard instance, create one by running the following cell:

In [16]:
# TODO 2
tensorboard = # TODO 2: Your code goes here

Creating Tensorboard
Create Tensorboard backing LRO: projects/21417957250/locations/us-central1/tensorboards/5219117814262005760/operations/6713704104704606208
Tensorboard created. Resource name: projects/21417957250/locations/us-central1/tensorboards/5219117814262005760
To use this Tensorboard in another session:
tb = aiplatform.Tensorboard('projects/21417957250/locations/us-central1/tensorboards/5219117814262005760')


Once your TensorBoard instance is created, you get the TensorBoard instance name.

In [17]:
TENSORBOARD_INSTANCE_NAME = tensorboard.resource_name

print("TensorBoard display name:", TENSORBOARD_NAME)
print("TensorBoard instance name:", TENSORBOARD_INSTANCE_NAME)

TensorBoard display name: tensorboard-tb
TensorBoard instance name: projects/21417957250/locations/us-central1/tensorboards/5219117814262005760


## Train a model

To train a model using your custom training code, choose one of the following options:

- **Prebuilt container**: Load your custom training code as a Python package to a prebuilt container image from Google Cloud.

- **Custom container**: Create your own container image that contains your custom training code.

In this tutorial, we will train a custom model using a Google Cloud prebuild container.

### Examine the training package

#### Package layout

Before you start the training, let's take a look at how a Python package is assembled for a custom training job. When extracted, the package contains the following:

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

The files `setup.cfg` and `setup.py` are the instructions for installing the package into the operating environment of the docker image.

The file `trainer/task.py` is the Python script for executing the custom training job. Notice that when referring to this file in worker_pool_spec, we replace the directory slash with a dot (`trainer.task`) and dropped the file suffix (`.py`).


In [18]:
PYTHON_PACKAGE_APPLICATION_DIR = "custom"

source_package_file_name = f"{PYTHON_PACKAGE_APPLICATION_DIR}/dist/trainer-0.1.tar.gz"
python_package_gcs_uri = f"{BUCKET_URI}/trainer-0.1.tar.gz"

In [19]:
# Make folder for Python training script
! rm -rf {PYTHON_PACKAGE_APPLICATION_DIR}
! mkdir {PYTHON_PACKAGE_APPLICATION_DIR}

# Add package information
! touch {PYTHON_PACKAGE_APPLICATION_DIR}/README.md

# Make the training subfolder
! mkdir {PYTHON_PACKAGE_APPLICATION_DIR}/trainer
! touch {PYTHON_PACKAGE_APPLICATION_DIR}/trainer/__init__.py

In [20]:
%%writefile ./{PYTHON_PACKAGE_APPLICATION_DIR}/setup.py

from setuptools import find_packages
from setuptools import setup
import setuptools

from distutils.command.build import build as _build
import subprocess

REQUIRED_PACKAGES = [
    'google-cloud-aiplatform[cloud_profiler]',
]

setup(
    install_requires=REQUIRED_PACKAGES,
    packages=find_packages(),
    include_package_data=True,
    name='trainer',
    version='0.1',
    url="wwww.google.com",
    description='Vertex AI | Training | Python Package'
)


Writing ./custom/setup.py


### Prepare the training script

Your training code must be configured to write TensorBoard logs to a Cloud Storage bucket, the location of which Vertex AI Training automatically makes available through a predefined environment variable, `AIP_TENSORBOARD_LOG_DIR`.

This can usually be done by providing `os.environ['AIP_TENSORBOARD_LOG_DIR']` as the log directory to the open source TensorBoard log writing APIs. 

For example, in TensorFlow 2.x, you can use following code to create a tensorboard_callback: 

    tensorboard_callback = tf.keras.callbacks.TensorBoard( 
      log_dir=os.environ['AIP_TENSORBOARD_LOG_DIR'], 
      histogram_freq=1) 
`AIP_TENSORBOARD_LOG_DIR` is in the `BASE_OUTPUT_DIR` that you provide when creating the custom training job.

To enable Vertex AI TensorBoard Profiler for your training job, add the following to your training script:

Add the cloud_profiler import at your top level imports:

    from google.cloud.aiplatform.training_utils import cloud_profiler


Initialize the cloud_profiler plugin by adding:


    cloud_profiler.init()

In [21]:
%%writefile ./{PYTHON_PACKAGE_APPLICATION_DIR}/trainer/task.py
#!/usr/bin/env python

import tensorflow as tf
import argparse
import os
from google.cloud.aiplatform.training_utils import cloud_profiler
import time

"""Train an mnist model and use cloud_profiler for profiling."""

def _create_model():
    model = tf.keras.models.Sequential(
        [
            tf.keras.layers.Flatten(input_shape=(28, 28)),
            tf.keras.layers.Dense(128, activation="relu"),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(10),
        ]
    )
    return model


def main(args):
    strategy = None
    if args.distributed:
        strategy = tf.distribute.MultiWorkerMirroredStrategy()

    mnist = tf.keras.datasets.mnist

    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0

    if args.distributed:
        strategy = tf.distribute.MultiWorkerMirroredStrategy()
        with strategy.scope():
            model = _create_model()
            model.compile(
                optimizer="adam",
                loss=tf.keras.losses.sparse_categorical_crossentropy,
                metrics=["accuracy"],
            )
    else:
        model = _create_model()
        model.compile(
            optimizer="adam",
            loss=tf.keras.losses.sparse_categorical_crossentropy,
            metrics=["accuracy"],
        )

    # Initialize the profiler.
    cloud_profiler.init()

    # Use AIP_TENSORBOARD_LOG_DIR to update where logs are written to.
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
       log_dir=os.environ["AIP_TENSORBOARD_LOG_DIR"], histogram_freq=1
    )

    model.fit(
        x_train,
        y_train,
        epochs=args.epochs,
        verbose=0,
        callbacks=[tensorboard_callback],
    )
    
    MODEL_DIR = os.getenv("AIP_MODEL_DIR")
    
    model.save(MODEL_DIR)

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--epochs", type=int, default=100, help="Number of epochs to run model."
    )
    parser.add_argument(
        "--distributed", action="store_true", help="Use MultiWorkerMirroredStrategy"
    )
    args = parser.parse_args()
    main(args)

Writing ./custom/trainer/task.py


Run the following command to create a source distribution, dist/trainer-0.1.tar.gz:

In [22]:
!cd {PYTHON_PACKAGE_APPLICATION_DIR} && python3 setup.py sdist --formats=gztar

running sdist
running egg_info
creating trainer.egg-info
writing trainer.egg-info/PKG-INFO
writing dependency_links to trainer.egg-info/dependency_links.txt
writing requirements to trainer.egg-info/requires.txt
writing top-level names to trainer.egg-info/top_level.txt
writing manifest file 'trainer.egg-info/SOURCES.txt'
reading manifest file 'trainer.egg-info/SOURCES.txt'
writing manifest file 'trainer.egg-info/SOURCES.txt'
running check

creating trainer-0.1
creating trainer-0.1/trainer
creating trainer-0.1/trainer.egg-info
copying files to trainer-0.1...
copying README.md -> trainer-0.1
copying setup.py -> trainer-0.1
copying trainer/__init__.py -> trainer-0.1/trainer
copying trainer/task.py -> trainer-0.1/trainer
copying trainer.egg-info/PKG-INFO -> trainer-0.1/trainer.egg-info
copying trainer.egg-info/SOURCES.txt -> trainer-0.1/trainer.egg-info
copying trainer.egg-info/dependency_links.txt -> trainer-0.1/trainer.egg-info
copying trainer.egg-info/requires.txt -> trainer-0.1/trainer.

Now upload the source distribution with training application to Cloud Storage bucket

In [23]:
# TODO 3
# Copy the source distribution to Cloud Storage bucket
# TODO 3: Your code goes here

Copying file://custom/dist/trainer-0.1.tar.gz [Content-Type=application/x-tar]...
/ [1 files][  1.7 KiB/  1.7 KiB]                                                
Operation completed over 1 objects/1.7 KiB.                                      


Validate the source distribution exists on Cloud Storage bucket

In [24]:
!gsutil ls -l {python_package_gcs_uri}

      1729  2022-10-14T10:22:14Z  gs://qwiklabs-gcp-00-1fe7a1835a07/trainer-0.1.tar.gz
TOTAL: 1 objects, 1729 bytes (1.69 KiB)


## Create and run the custom training job

Configure a [custom job](https://cloud.google.com/vertex-ai/docs/training/create-custom-job) with the [pre-built container](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers) image for training code packaged as Python source distribution. 

**NOTE:** When using Vertex AI SDK for Python for submitting a training job, it creates a [training pipeline](https://cloud.google.com/vertex-ai/docs/training/create-training-pipeline) which launches the custom job on Vertex AI Training service.

In [25]:
JOB_NAME = "custom_job_" + TIMESTAMP
MACHINE_TYPE = "n1-standard-4"

base_output_dir = "{}/{}".format(BUCKET_URI, JOB_NAME)
python_module_name = "trainer.task"

EPOCHS = 20
training_args = [
    "--epochs=" + str(EPOCHS),
]

In [28]:
print(f"JOB_NAME={JOB_NAME}")
print(f"python_module_name={python_module_name}")
print(f"PRE_BUILT_TRAINING_CONTAINER_IMAGE_URI={TRAIN_IMAGE}")
print(f"python_package_gcs_uri={python_package_gcs_uri}")
print(f"base_output_dir={base_output_dir}")
print(f"tensorboard={TENSORBOARD_INSTANCE_NAME}")
print(f"service_account={SERVICE_ACCOUNT}")

JOB_NAME=custom_job_20221014101628
python_module_name=trainer.task
PRE_BUILT_TRAINING_CONTAINER_IMAGE_URI=us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-9:latest
python_package_gcs_uri=gs://qwiklabs-gcp-00-1fe7a1835a07/trainer-0.1.tar.gz
base_output_dir=gs://qwiklabs-gcp-00-1fe7a1835a07/custom_job_20221014101628
tensorboard=projects/21417957250/locations/us-central1/tensorboards/5219117814262005760
service_account=21417957250-compute@developer.gserviceaccount.com


In [29]:
# TODO 4
# Create a custom training job
job = aiplatform.CustomPythonPackageTrainingJob(
# TODO 4: Your code goes here
)

In [30]:
if TRAIN_GPU:
    job.run(
        args=training_args,
        replica_count=1,
        machine_type=MACHINE_TYPE,
        accelerator_type=TRAIN_GPU.name,
        accelerator_count=TRAIN_NGPU,
        base_output_dir=base_output_dir,
        tensorboard=TENSORBOARD_INSTANCE_NAME,
        service_account=SERVICE_ACCOUNT,
        sync=True,
    )
else:
    job.run(
        args=training_args,
        replica_count=1,
        machine_type=MACHINE_TYPE,
        base_output_dir=base_output_dir,
        tensorboard=TENSORBOARD_INSTANCE_NAME,
        service_account=SERVICE_ACCOUNT,
        sync=True,
    )

Training Output directory:
gs://qwiklabs-gcp-00-1fe7a1835a07/custom_job_20221014101628 
View Training:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/7902914372110909440?project=21417957250
CustomPythonPackageTrainingJob projects/21417957250/locations/us-central1/trainingPipelines/7902914372110909440 current state:
PipelineState.PIPELINE_STATE_PENDING
CustomPythonPackageTrainingJob projects/21417957250/locations/us-central1/trainingPipelines/7902914372110909440 current state:
PipelineState.PIPELINE_STATE_RUNNING
View backing custom job:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/7968119809684537344?project=21417957250
View tensorboard:
https://us-central1.tensorboard.googleusercontent.com/experiment/projects+21417957250+locations+us-central1+tensorboards+5219117814262005760+experiments+7968119809684537344
CustomPythonPackageTrainingJob projects/21417957250/locations/us-central1/trainingPipelines/7902914372110909440 current s

## View the TensorBoard Profiler dashboard


When the custom job state switches to `Running`, you can access the Vertex AI TensorBoard Profiler dashboard through the Custom jobs page or the Experiments page on the Google Cloud console. 

The Google Cloud guide to [Profile model training performance using Profiler](https://cloud.google.com/vertex-ai/docs/experiments/tensorboard-profiler) provides detailed instructions for accessing the Vertex AI TensorBoard Profiler dashboard and capturing a profiling session. 


## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

- Training job
- TensorBoard instance
- Cloud Storage bucket


In [31]:
delete_customjob = True
delete_tensorboard = True
delete_bucket = True

try:
    if delete_customjob:
        job.delete()
except Exception as e:
    print(e)

try:
    if delete_tensorboard:
        tensorboard.delete()
except Exception as e:
    print(e)

if delete_bucket and "BUCKET_URI" in globals():
    ! gsutil -m rm -r $BUCKET_URI

Deleting CustomPythonPackageTrainingJob : projects/21417957250/locations/us-central1/trainingPipelines/7902914372110909440
Delete CustomPythonPackageTrainingJob  backing LRO: projects/21417957250/locations/us-central1/operations/7150553268559544320
CustomPythonPackageTrainingJob deleted. . Resource name: projects/21417957250/locations/us-central1/trainingPipelines/7902914372110909440
Deleting Tensorboard : projects/21417957250/locations/us-central1/tensorboards/5219117814262005760
Delete Tensorboard  backing LRO: projects/21417957250/locations/us-central1/operations/4776030365028450304
Tensorboard deleted. . Resource name: projects/21417957250/locations/us-central1/tensorboards/5219117814262005760
Removing gs://qwiklabs-gcp-00-1fe7a1835a07/trainer-0.1.tar.gz#1665742934265426...
Removing gs://qwiklabs-gcp-00-1fe7a1835a07/custom_job_20221014101628/logs/train/#1665743342029476...
Removing gs://qwiklabs-gcp-00-1fe7a1835a07/custom_job_20221014101628/#1665743341627028...
Removing gs://qwikla